In [1]:
import pandas as pd
import scipy as sp

In [2]:
gobackground = pd.read_table('/home/ariel/Projects/Gutierrez/EBV-recomb/recomb/GOanalysis/GO-EBV-Uniprot.tab')

In [3]:
mygentable = pd.read_table('/data/EBV/by_gene/gene_result.txt')


In [4]:
gobackground['Cross-reference (GeneID)'][9]

'3783696;3783733;'

In [5]:
gids = gobackground['Cross-reference (GeneID)'].str.split(';',expand=False).apply(lambda x: [a  for a in x if a not in ''])
gidslist = pd.to_numeric(pd.concat([pd.Series(x) for x in gids.values]))
gidframe = gidslist.reset_index().drop(['index'],axis=1)

In [6]:
mygentable.GeneID.head()

0     3783676
1     3783677
2     3783678
3    17494228
4     3783679
Name: GeneID, dtype: int64

In [7]:
gidframe.head()

,0
0,3783746
1,17494212
2,3783750
3,3783713
4,3783761


In [8]:
# All uniprots IDs are in our gene table
print gidframe[0].isin(mygentable.GeneID).all()

# The inverse is not true
print mygentable.GeneID.isin(gidframe[0]).all()

print mygentable.GeneID.isin(gidframe[0]).sum()/float(mygentable.shape[0])

print 'some examples not present in uniprot -- all of them are 5~6 bp long'
print mygentable.GeneID[~mygentable.GeneID.isin(gidframe[0])].head()

True
False
0.59477124183
some examples not present in uniprot -- all of them are 5~6 bp long
3     17494228
7     17494205
11    17494226
13    17494249
16    17494233
Name: GeneID, dtype: int64


In [9]:
#ggids = [pd.Series(int(g[0])) if len(g)==1 else g.to_frame().transpose() for g in gids]
ggids = [int(g[0]) for g in gids]
gobackground['GeneID'] = ggids

In [10]:
merged = pd.merge(gobackground,mygentable.drop(['Status'],axis = 1),on = ['GeneID'])

### godict


In [11]:
definit = gobackground['Gene ontology (GO)'].str.split(';',expand = False)
ddef =  []
for go in definit:
    if type(go)== list:
        ddef = ddef + go
    else:
        ddef = ddef + [go]
ddef = pd.Series(ddef).drop_duplicates()


ddef = ddef.str.split(' \[')
ddef =pd.Series([x for x in ddef if type(x)==list])

ddefs = ddef.apply(lambda x: pd.Series([x[0],x[1]]))
ddefs.columns =['def','goid']
ddefs['goid'] = ddefs.goid.str.split('\]').apply(lambda x:x[0])

#ddef.apply(lambda x: type(x)).value_counts()
#xx = ddef.apply(lambda x: {'name':x[0],'iid':x[1]})
#xx
#pd.DataFrame(xx)

In [12]:
godict = {}
for i in range(ddefs.shape[0]):
    x = ddefs.iloc[i,:]
    godict.update({x['goid'].replace(' ',''):x['def']})

In [13]:
by_repeats = pd.read_csv('~/Projects/Gutierrez/EBV-recomb/recomb/rdp4_results/ids_171_ebv/10Miter/results/filtered_gene_vs_repeat.csv')
by_repeats.GENE.nunique()              


3

In [14]:
by_motifs = pd.read_csv('/home/ariel/Projects/Gutierrez/EBV-recomb/recomb/rdp4_results/ids_171_ebv/10Miter/results/filtered_gene_vs_call.csv')
by_motifs.GENE.nunique()

28

In [66]:
by_gene_intensity = ['EBNA3C',
 'EBNA3B',
 'BRRF2',
 'BZLF2',
 'BBLF2-BBLF3',
 'BLLF2',
 'BZLF1',
 'BMRF2',
 'BBRF2',
 'BMLF1',
 'EBNA3A',
 'EBNA1 leader sequence',
 'LMP2B',
 'SM, spliced BSLF2+BMLF1',
 'BERF3',
 'BORF2',
 'BMRF1',
 'BDLF4',
 'BKRF3',
 'LMP1']

In [180]:
import pickle
outpath = '/home/ariel/Projects/Gutierrez/EBV-recomb/recomb/rdp4_results/ids_171_ebv/10Miter/results/'
with open(outpath+'motif_gene_dict.pickle', 'rb') as handle:
    motif_gene_dict = pickle.load(handle)
motifs = motif_gene_dict.keys()

m =  motifs[5]
print m
by_gene_intensity = motif_gene_dict[m]


TGGTGG


In [206]:
l = motif_gene_dict.values()
flat_list = [item for sublist in l for item in sublist]
by_gene_intensity = pd.Series(flat_list).unique().tolist()

In [207]:
#goserie = merged[merged.Symbol.isin(by_motifs.GENE)]['Gene ontology IDs'].str.split(';',expand=False)
goserie = merged[merged.Symbol.isin(by_gene_intensity)]['Gene ontology IDs'].str.split(';',expand=False)

goes =  []
for go in goserie:
    if type(go)== list:
        goes = goes + go
    else:
        goes = goes + [go]
goes = pd.Series(goes).str.replace(' ','')
top_goes = goes.value_counts()


In [208]:
aux = merged[['Gene ontology IDs','Symbol']].copy()
aux.set_index(['Symbol'],inplace = True)
dic = aux['Gene ontology IDs'].str.split('; ',expand = False)

In [209]:
import numpy as np
inv_map = {}
for k, v in dic.iteritems():
    if type(v)!=float:
        for vi in v:
            inv_map[vi] = inv_map.get(vi, [])
            inv_map[vi].append(k)

In [210]:
counts ={}
for k in inv_map.keys():
    counts.update({k:len(inv_map[k])})
gocounts = pd.Series(counts).sort_values(ascending = False)

## contingency table:

In [211]:
total_genes = aux.index.nunique()  # total pelotitas en la bolsa
total_genes_with_signif_rho =  by_motifs.GENE.nunique()    # total de pelotitas blancas

N = total_genes
bcas = total_genes_with_signif_rho

In [212]:
import scipy.stats as stats

fisher = pd.DataFrame()
for go in counts.keys():
    marginal_go = counts[go]
    if go in top_goes.keys():
        go_hit = top_goes[go]
    else:
        go_hit = 0
        
    go_no_hit = marginal_go -go_hit
    no_go_hit = total_genes_with_signif_rho - go_hit
    no_go_no_hit = (total_genes -total_genes_with_signif_rho) - go_no_hit

    m = np.array([[go_hit,go_no_hit],[no_go_hit,no_go_no_hit]])
    oddsratio, pvalue = stats.fisher_exact(m,alternative='greater')
    fisher = pd.concat([fisher,pd.Series([go,pvalue,go_hit,marginal_go,godict[go]])],axis = 1)
fisher = fisher.transpose()
fisher.columns = ['go','pval','hits','marginal_go','description']
fisher.sort_values(by = 'pval',inplace = True)

In [213]:
annotations = pd.Series(inv_map).to_frame()
annotations.columns = ['genes']



In [214]:
annotations.head()

,genes
GO:0000166,[BALF5]
GO:0003677,"[LMP2, BMRF1, BKRF1, BZLF1, BALF5, BGLF5, BLRF..."
GO:0003688,[BKRF1]
GO:0003697,[BALF2]
GO:0003700,"[BKRF1, BZLF1]"


In [215]:
fish = pd.merge(fisher,annotations,right_index = True,left_on='go',how = 'left')
fish.set_index(['go'],inplace = True)
fish.head()

,pval,hits,marginal_go,description,genes
go,,,,,
GO:0039686,0.0010759,6,6,bidirectional double-stranded viral DNA replic...,"[BMRF1, BALF5, BALF2, BSLF1, BBLF2-BBLF3, BBLF4]"
GO:0042025,0.00939659,16,31,host cell nucleus,"[EBNA-LP, LMP2, LMP2, BYRF1, BMRF1, BKRF1, BdR..."
GO:0006260,0.0162321,5,6,DNA replication,"[BKRF1, BALF5, BALF2, BaRF1, BORF2, BBLF4]"
GO:0006323,0.0369919,3,3,DNA packaging,"[LMP2, LMP2, BGLF1]"
GO:0003677,0.0716816,6,10,DNA binding,"[LMP2, BMRF1, BKRF1, BZLF1, BALF5, BGLF5, BLRF..."


In [216]:
oo = fish.sort_values(by=['hits'],ascending = False)
included = oo.genes.apply(lambda x: [i for i in x if i in by_gene_intensity ])
included = included.to_frame()
included.columns =['included']

In [219]:
rr = pd.concat([fish.sort_values(by=['hits'],ascending = False),included],axis = 1)
rr[rr.hits>1]

,pval,hits,marginal_go,description,genes,included
go,,,,,,
GO:0042025,0.00939659,16,31,host cell nucleus,"[EBNA-LP, LMP2, LMP2, BYRF1, BMRF1, BKRF1, BdR...","[LMP2, LMP2, BMRF1, BdRF1, BGLF4, BMLF1, BALF5..."
GO:0019033,0.0780488,8,15,viral tegument,"[BMRF1, BGLF4, BPLF1, BFRF2, BKRF4, BRRF2, BLR...","[BMRF1, BGLF4, BRRF2, BLRF2, BBRF2, BALF2, BNR..."
GO:0039686,0.0010759,6,6,bidirectional double-stranded viral DNA replic...,"[BMRF1, BALF5, BALF2, BSLF1, BBLF2-BBLF3, BBLF4]","[BMRF1, BALF5, BALF2, BSLF1, BBLF2-BBLF3, BBLF4]"
GO:0003677,0.0716816,6,10,DNA binding,"[LMP2, BMRF1, BKRF1, BZLF1, BALF5, BGLF5, BLRF...","[LMP2, BMRF1, BALF5, BGLF5, BLRF2, BRLF1]"
GO:0006260,0.0162321,5,6,DNA replication,"[BKRF1, BALF5, BALF2, BaRF1, BORF2, BBLF4]","[BALF5, BALF2, BaRF1, BORF2, BBLF4]"
GO:0016021,0.770239,5,17,integral component of membrane,"[LMP2, LMP1, BLLF1, BZLF2, BALF4, BHRF1, BMRF2...","[LMP2, BLLF1, BMRF2, BBRF3, BaRF1]"
GO:0006351,0.299367,5,11,"transcription, DNA-templated","[EBNA-LP, LMP2, BYRF1, BMRF1, BKRF1, BZLF1, BM...","[LMP2, BMRF1, BMLF1, EBNA3B, BRLF1]"
GO:0006355,0.144326,5,9,"regulation of transcription, DNA-templated","[EBNA-LP, LMP2, BMRF1, BZLF1, BMLF1, EBNA3C, B...","[LMP2, BMRF1, BMLF1, EBNA3B, BRLF1]"
GO:0046872,0.0999069,4,6,metal ion binding,"[BMLF1, BLLF3, BFLF2, BSLF1, BaRF1, BALF3]","[BMLF1, BLLF3, BSLF1, BaRF1]"


In [70]:
fish.to_csv('/home/ariel/Projects/Gutierrez/EBV-recomb/recomb/GOanalysis/fisher_test.tsv',sep ='\t',index = True)

In [24]:
#for i in range(52):
#    print godict[top_goes.index[i]]
    

In [25]:
ddefs[ddefs['def'].str.contains('repair')]

,def,goid
129,base-excision repair,GO:0006284


In [26]:
merged[merged['Gene ontology IDs'].str.contains('GO:0006284').fillna(False)].Symbol

72    BKRF3
Name: Symbol, dtype: object

In [39]:
#gobackground.rename(columns ={'Cross-reference (GeneID)':'GeneID'},inplace = True)

In [56]:
gobackground.columns

Index([u'Entry', u'Entry name', u'Status', u'Protein names', u'Length',
       u'Gene ontology (biological process)',
       u'Gene ontology (cellular component)',
       u'Gene ontology (molecular function)', u'Gene ontology IDs',
       u'Gene ontology (GO)', u'Gene names', u'Cross-reference (GeneID)',
       u'GeneID'],
      dtype='object')

In [67]:
#gobackground.GeneID.str.split(',')

In [109]:
merged[merged.Symbol.isin(['BMRF2'])]

,Entry,Entry name,Status,Protein names,Length,Gene ontology (biological process),Gene ontology (cellular component),Gene ontology (molecular function),Gene ontology IDs,Gene ontology (GO),...,other_designations,map_location,chromosome,genomic_nucleotide_accession.version,start_position_on_the_genomic_accession,end_position_on_the_genomic_accession,orientation,exon_count,OMIM,Unnamed: 17
23,P03192,BMRF2_EBVB9,reviewed,Protein BMRF2,357,viral entry into host cell [GO:0046718]; virio...,host cell plasma membrane [GO:0020002]; integr...,NaN,GO:0016021; GO:0019062; GO:0020002; GO:0046718...,host cell plasma membrane [GO:0020002]; integr...,...,NaN,NaN,NaN,NC_007605.1,68491.0,69903.0,plus,0.0,NaN,NaN
